### Connectivity of cities in Europe

In [95]:
import pandas as pd

# for connectivity there's required only one matrice with minimum time to different cities
trains = pd.read_csv("../datasets/train_distances3/combined_train_distances3.csv")
flights = pd.read_csv("../datasets/final_flight_routes.csv")
merged_distance = pd.read_csv("../datasets/merged_df.csv")

trains

,departure city,arrival city,transit_distance [m],transit_time [s]
0,Arrecife,Barcelona,-1,-1
1,Alicante,Barcelona,525005,19080
2,Amsterdam,Barcelona,1627336,44340
3,Athens,Barcelona,-1,-1
4,Málaga,Barcelona,1110453,22380
...,...,...,...,...
1333,Vienna,Santander,-1,-1
1334,Vienna,Tromsø,3970237,197151
1335,Moscow,Groznyy,3147441,126600
1336,Moscow,Samara,1569205,52860


In [96]:
# transform train connections
trains = trains.loc[~(trains["transit_time [s]"] == -1)]
trains["time"] = trains["transit_time [s]"].apply(lambda x: x//60)
trains.drop(columns=["transit_distance [m]","transit_time [s]"], inplace=True)
trains.rename(columns={"departure city":"city1",
                       "arrival city":"city2"}, inplace=True)


C:\Users\Adam\AppData\Local\Temp\ipykernel_19484\3866892317.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trains["time"] = trains["transit_time [s]"].apply(lambda x: x//60)
C:\Users\Adam\AppData\Local\Temp\ipykernel_19484\3866892317.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trains.drop(columns=["transit_distance [m]","transit_time [s]"], inplace=True)
C:\Users\Adam\AppData\Local\Temp\ipykernel_19484\3866892317.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: htt

In [97]:
trains["route"] = trains.apply(lambda row: str( set([row["city1"], row["city2"]]) ), axis=1)
trains.drop(columns=["city1","city2"], inplace=True)

flights.drop(columns=["route","departure airport","arrival airport"], inplace=True)
flights.rename(columns={"departure city":"city1",
                       "arrival city":"city2"}, inplace=True)
flights["route"] = flights.apply(lambda row: str( set([row["city1"], row["city2"]]) ), axis=1)
flights.drop(columns=["city1","city2"], inplace=True)


C:\Users\Adam\AppData\Local\Temp\ipykernel_19484\2419805734.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trains["route"] = trains.apply(lambda row: str( set([row["city1"], row["city2"]]) ), axis=1)
C:\Users\Adam\AppData\Local\Temp\ipykernel_19484\2419805734.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trains.drop(columns=["city1","city2"], inplace=True)


In [98]:
flights["time60"] = flights["time"] + 60
flights["time45"] = flights["time"] + 45
flights["time90"] = flights["time"] + 90 
flights["time120"] = flights["time"] + 120
flights

,time,route,time60,time45,time90,time120
0,85.0,"{'Aalborg', 'Amsterdam'}",145.0,130.0,175.0,205.0
1,170.0,"{'Saint Petersburg', 'Novorossiysk'}",230.0,215.0,260.0,290.0
2,85.0,"{'Amsterdam', 'Aberdeen'}",145.0,130.0,175.0,205.0
3,95.0,"{'London', 'Aberdeen'}",155.0,140.0,185.0,215.0
4,130.0,"{'Arrecife', 'Málaga'}",190.0,175.0,220.0,250.0
...,...,...,...,...,...,...
1333,150.0,"{'Zürich', 'Bucharest'}",210.0,195.0,240.0,270.0
1334,127.5,"{'Zürich', 'Pristina'}",187.5,172.5,217.5,247.5
1335,205.0,"{'Zürich', 'Rovaniemi'}",265.0,250.0,295.0,325.0
1336,45.0,"{'Zürich', 'Stuttgart'}",105.0,90.0,135.0,165.0


In [99]:
trains

,time,route
1,318,"{'Barcelona', 'Alicante'}"
2,739,"{'Amsterdam', 'Barcelona'}"
4,373,"{'Málaga', 'Barcelona'}"
6,380,"{'Bordeaux', 'Barcelona'}"
7,627,"{'Brussels', 'Barcelona'}"
...,...,...
1331,280,"{'Vienna', 'Klagenfurt'}"
1332,599,"{'Vienna', 'Trier'}"
1334,3285,"{'Vienna', 'Tromsø'}"
1335,2110,"{'Moscow', 'Groznyy'}"


In [100]:
def calculate_closeness(trains, flights, flight_time="time"):
    flights = flights[["route",flight_time]].copy()
    print(flights.columns)
    flights.rename({flight_time:"time"},inplace=True)
    
    DF = pd.concat([trains, flights])
    DF = DF.groupby(["route"]).min()
    DF.reset_index(inplace=True)
    DF[["city1","city2"]] = DF["route"].apply(lambda x: pd.Series(list(eval(x))))
    DF.drop(columns=["route"],inplace=True )
    DF["time"] = 1/DF["time"]

    # Combine unique cities from both columns into a single set
    all_cities = list(set(DF['city1']).union(set(DF['city2'])))

    # Create a DataFrame with all unique cities as both index and columns
    distance_matrix = pd.DataFrame(index=all_cities, columns=all_cities)

    distance_matrix.values[[range(len(all_cities))]*2] = 0
    # Fill the distance matrix with distances from the original DataFrame
    for _, row in DF.iterrows():
        city1, city2, time = row['city1'], row['city2'], row['time']
        distance_matrix.at[city1, city2] = time
        distance_matrix.at[city2, city1] = time  # Assuming distances are symmetric

    # Fill diagonal with zeros (distance from a city to itself)
    closeness_cities = pd.DataFrame({
        "City":all_cities,
        "Closeness":distance_matrix.sum(axis=1)
    })
    return closeness_cities.reset_index().iloc[:,1:]

In [101]:
closeness = calculate_closeness(trains, flights)
closeness.sort_values("Closeness",ascending=False).iloc[:10,:]

Index(['route', 'time'], dtype='object')


,City,Closeness
65,London,0.784561
225,Frankfurt,0.585549
137,Amsterdam,0.541825
111,Rome,0.538658
156,Munich,0.509447
140,Madrid,0.490417
116,Barcelona,0.477464
9,Paris,0.477409
90,Athens,0.378658
231,Vienna,0.366777


In [102]:
closeness = calculate_closeness(trains, flights, "time45")
closeness.sort_values("Closeness",ascending=False).iloc[:10,:]

Index(['route', 'time45'], dtype='object')


,City,Closeness
225,Frankfurt,0.145137
65,London,0.131604
9,Paris,0.114955
111,Rome,0.103805
140,Madrid,0.098245
156,Munich,0.089299
137,Amsterdam,0.079304
85,Brussels,0.059046
116,Barcelona,0.055977
93,Zürich,0.054904


In [103]:
closeness = calculate_closeness(trains, flights, "time60")
closeness.sort_values("Closeness",ascending=False).iloc[:10,:]

Index(['route', 'time60'], dtype='object')


,City,Closeness
225,Frankfurt,0.145137
65,London,0.131604
9,Paris,0.114955
111,Rome,0.103805
140,Madrid,0.098245
156,Munich,0.089299
137,Amsterdam,0.079304
85,Brussels,0.059046
116,Barcelona,0.055977
93,Zürich,0.054904


In [104]:
closeness = calculate_closeness(trains, flights, "time90")
closeness.sort_values("Closeness",ascending=False).iloc[:10,:]

Index(['route', 'time90'], dtype='object')


,City,Closeness
225,Frankfurt,0.145137
65,London,0.131604
9,Paris,0.114955
111,Rome,0.103805
140,Madrid,0.098245
156,Munich,0.089299
137,Amsterdam,0.079304
85,Brussels,0.059046
116,Barcelona,0.055977
93,Zürich,0.054904
